In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup 
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import json
import random

In [2]:
def get_browser():
    return Browser('chrome')

browser = get_browser()
target_city = "delhi"
output_file = f"car_links_{target_city}.txt"
url = f"https://www.cardekho.com/used-cars+in+{target_city}"

try:
    browser.visit(url)
    time.sleep(3) # Initial load

    extracted_links = set()
    no_change_count = 0

    while no_change_count < 3:
        previous_count = len(extracted_links)
        
        # 1. Faster, snappy scrolling
        # We use a smaller sleep (0.5 to 0.8) between small scrolls
        for _ in range(8):
            browser.execute_script("window.scrollBy(0, 1500);")
            time.sleep(random.uniform(0.5, 0.8)) 
        
        # 2. Faster "View More" check
        # .is_element_present_by_text with a low wait_time is very fast
        if browser.is_element_present_by_text('View More Cars', wait_time=0.5):
            try:
                browser.find_by_text('View More Cars').click()
                # Short wait after click for the DOM to update
                time.sleep(1.5) 
            except:
                pass

        # 3. Quick Extraction
        page_soup = soup(browser.html, 'html.parser')
        car_containers = page_soup.find_all('div', class_=['titlebox', 'hover'])
        
        for container in car_containers:
            link_tag = container.find('a', href=True)
            if link_tag:
                full_link = f"https://www.cardekho.com{link_tag['href']}".split('?')[0]
                if "/used-car-details/" in full_link and full_link not in extracted_links:
                    extracted_links.add(full_link)
                    with open(output_file, "a") as f:
                        f.write(full_link + "\n")

        current_count = len(extracted_links)
        print(f"Collected: {current_count}")

        # 4. Smart Exit
        if current_count > previous_count:
            no_change_count = 0
        else:
            no_change_count += 1
            # If no progress, wait a tiny bit longer once to see if it's just lag
            time.sleep(2)

except Exception as e:
    print(f"Error: {e}")
finally:
    browser.quit()
    print(f"Done! Final unique links: {len(extracted_links)}")

Collected: 58
Collected: 92
Collected: 140
Collected: 173
Collected: 225
Collected: 283
Collected: 345
Collected: 398
Collected: 452
Collected: 510
Collected: 552
Collected: 598
Collected: 649
Collected: 703
Collected: 748
Collected: 799
Collected: 829
Collected: 885
Collected: 928
Collected: 976
Collected: 1024
Collected: 1068
Collected: 1101
Collected: 1101
Collected: 1101
Collected: 1101
Done! Final unique links: 1101


---

In [15]:
def get_browser():
    return Browser('chrome')

# 1. Load your links
with open("car_links.txt", "r") as f:
    all_links = [line.strip() for line in f.readlines()]

browser = get_browser()
output_file = "car_dataset_kolkata.json"

for index, link in enumerate(all_links):
    try:
        print(f"Scraping {index+1}/{len(all_links)}: {link}")
        browser.visit(link)
        
        # --- Adjusted Timing ---
        # Increased initial wait to 1.5-2s to ensure the button and price load
        time.sleep(random.uniform(1, 1.5))
        browser.execute_script("window.scrollTo(0, 600);")
        time.sleep(0.5)

        # 2. Expansion Logic
        try:
            view_all_spec_btn = browser.find_by_text('View all Specifications')
            if view_all_spec_btn:
                browser.execute_script("arguments[0].click();", view_all_spec_btn.first._element)
                # Reduced wait slightly to 1.5s for speed, usually enough for text
                time.sleep(1.5) 
        except Exception:
            pass

        # 3. Scrape with BeautifulSoup
        page_soup = soup(browser.html, 'html.parser')
        car_data = {"url": link}
        
        # --- PRICE EXTRACTION ---
        # Targeting the div and span seen in your previous screenshots
        price_div = page_soup.find('div', class_='vehiclePrice')
        if price_div:
            price_span = price_div.find('span')
            if price_span:
                car_data["Price"] = price_span.get_text(strip=True)

        # 4. Extract Labels and Values
        spec_items = page_soup.find_all('li', class_='gsc_col-xs-12')
        for item in spec_items:
            label_tag = item.find('div', class_='label')
            value_tag = item.find('span', class_='value-text')
            if label_tag and value_tag:
                car_data[label_tag.get_text(strip=True)] = value_tag.get_text(strip=True)

        # 5. Safety Check & Save
        if len(car_data) > 2: # URL + Price + at least one spec
            with open(output_file, "a") as out:
                out.write(json.dumps(car_data) + "\n")
            print(f"Saved: {car_data.get('Price', 'N/A')} | {len(car_data)-2} specs.")
        else:
            print("WARNING: Data missing. Check if page loaded correctly.")

    except Exception as e:
        print(f"Error at {link}: {e}")
        browser.quit()
        browser = get_browser()
        time.sleep(1.5) 
        continue

browser.quit()

Scraping 1/945: https://www.cardekho.com/used-car-details/used-Maruti-s-presso-vxi-at-2019-2022-cars-Kolkata_728acab8-3205-4630-87eb-be39ee59fd85.htm
Saved: ₹3.50 Lakh | 15 specs.
Scraping 2/945: https://www.cardekho.com/used-car-details/used-Audi-a6-45-tfsi-premium-plus-cars-Kolkata_c53e8212-0990-45a8-bc86-50041e2aff8f.htm
Saved: ₹47 Lakh | 53 specs.
Scraping 3/945: https://www.cardekho.com/used-car-details/used-Mg-hector-sharp-cvt-cars-Kolkata_5e039e47-bf5c-4f1c-9418-430d98e1a78e.htm
Saved: ₹17.55 Lakh | 42 specs.
Scraping 4/945: https://www.cardekho.com/used-car-details/used-Kia-sonet-htx-turbo-imt-bsvi-cars-Kolkata_6c8f8258-da74-4792-9696-78d90ea45f91.htm
Saved: ₹8.25 Lakh | 42 specs.
Scraping 5/945: https://www.cardekho.com/used-car-details/used-Skoda-kushaq-10-tsi-style-4-airbags-at-cars-Kolkata_b6784866-9ed2-4a37-bbd4-64e6ae963f57.htm
Saved: ₹10.35 Lakh | 44 specs.
Scraping 6/945: https://www.cardekho.com/used-car-details/used-Tata-tiago-12-revotron-xz-cars-Kolkata_7bcf5281-53dc

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


Scraping 412/945: https://www.cardekho.com/used-car-details/used-Hyundai-alcazar-platinum-7-seater-cars-Kolkata_8b72f217-2181-45a6-a839-ad0fcfa07879.htm
Error at https://www.cardekho.com/used-car-details/used-Hyundai-alcazar-platinum-7-seater-cars-Kolkata_8b72f217-2181-45a6-a839-ad0fcfa07879.htm: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=143.0.7499.193)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff60f0288d5
	0x7ff60f028930
	0x7ff60ee0165d
	0x7ff60edfe1dd
	0x7ff60edeeb69
	0x7ff60edf0974
	0x7ff60edef10f
	0x7ff60edee871
	0x7ff60edee5bd
	0x7ff60edec20a
	0x7ff60edec9f2
	0x7ff60ee05950
	0x7ff60eeabad4
	0x7ff60ee81fda
	0x7ff60eeaac97
	0x7ff60ee4ac29
	0x7ff60ee4ba93
	0x7ff60f340620
	0x7ff60f33af60
	0x7ff60f3596c6
	0x7ff60f045dd4
	0x7ff60f04ed7c
	0x7ff60f031ff4
	0x7ff60f0321a5
	0x7ff60f017ed2
	0x7ffdbc59e8d7
	0x7ffdbda0c53c



There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Scraping 413/945: https://www.cardekho.com/used-car-details/used-Hyundai-verna-crdi-16-sx-option-cars-Kolkata_452beaf4-39e5-4503-9530-5238b78227d8.htm
Saved: ₹7.75 Lakh | 15 specs.
Scraping 414/945: https://www.cardekho.com/used-car-details/used-Hyundai-i10-era-cars-Kolkata_66e7446b-6f9c-4510-a2a9-f0e9ebae171c.htm
Error at https://www.cardekho.com/used-car-details/used-Hyundai-i10-era-cars-Kolkata_66e7446b-6f9c-4510-a2a9-f0e9ebae171c.htm: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=143.0.7499.193)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff60f0288d5
	0x7ff60f028930
	0x7ff60ee0165d
	0x7ff60edfe1dd
	0x7ff60edeeb69
	0x7ff60edf0974
	0x7ff60edef10f
	0x7ff60edee871
	0x7ff60edee5bd
	0x7ff60edec20a
	0x7ff60edec9f2
	0x7ff60ee05950
	0x7ff60eeabad4
	0x7ff60ee81fda
	0x7ff60eeaac97
	0x7ff60ee4ac29
	0x7ff60ee4ba93
	0x7ff60f340620
	0x7ff60f33af60
	0x7ff60f3596c6
	0x7ff60f045dd4
	0x7ff60f04ed7c
	0x7ff60f031ff4
	0x7ff60f0321a5
	0x7ff60f017e

...

Scraping 429/945: https://www.cardekho.com/used-car-details/used-Toyota-innova-crysta-24-vx-mt-8s-bsiv-cars-Kolkata_73b1d9c3-d0af-479e-874c-9a9dcf50193b.htm
Error at https://www.cardekho.com/used-car-details/used-Toyota-innova-crysta-24-vx-mt-8s-bsiv-cars-Kolkata_73b1d9c3-d0af-479e-874c-9a9dcf50193b.htm: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=143.0.7499.193)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff60f0288d5
	0x7ff60f028930
	0x7ff60ee0165d
	0x7ff60edfe1dd
	0x7ff60edeeb69
	0x7ff60edf0974
	0x7ff60edef10f
	0x7ff60edee871
	0x7ff60edee5bd
	0x7ff60edec20a
	0x7ff60edec9f2
	0x7ff60ee05950
	0x7ff60eeabad4
	0x7ff60ee81fda
	0x7ff60eeaac97
	0x7ff60ee4ac29
	0x7ff60ee4ba93
	0x7ff60f340620
	0x7ff60f33af60
	0x7ff60f3596c6
	0x7ff60f045dd4
	0x7ff60f04ed7c
	0x7ff60f031ff4
	0x7ff60f0321a5
	0x7ff60f017ed2
	0x7ffdbc59e8d7
	0x7ffdbda0c53c



There was an error managing chromedriver (error decoding response body); using driver found in the cache


Scraping 430/945: https://www.cardekho.com/used-car-details/used-Mahindra-scorpio-s11-bsiv-cars-Kolkata_e1cd7e77-7b41-4dd3-9376-658fb8c0be15.htm
Error at https://www.cardekho.com/used-car-details/used-Mahindra-scorpio-s11-bsiv-cars-Kolkata_e1cd7e77-7b41-4dd3-9376-658fb8c0be15.htm: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=143.0.7499.193)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff60f0288d5
	0x7ff60f028930
	0x7ff60ee0165d
	0x7ff60edfe1dd
	0x7ff60edeeb69
	0x7ff60edf0974
	0x7ff60edef10f
	0x7ff60edee871
	0x7ff60edee5bd
	0x7ff60edec20a
	0x7ff60edec9f2
	0x7ff60ee05950
	0x7ff60eeabad4
	0x7ff60ee81fda
	0x7ff60eeaac97
	0x7ff60ee4ac29
	0x7ff60ee4ba93
	0x7ff60f340620
	0x7ff60f33af60
	0x7ff60f3596c6
	0x7ff60f045dd4
	0x7ff60f04ed7c
	0x7ff60f031ff4
	0x7ff60f0321a5
	0x7ff60f017ed2
	0x7ffdbc59e8d7
	0x7ffdbda0c53c



There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Scraping 431/945: https://www.cardekho.com/used-car-details/used-Renault-duster-85ps-diesel-rxs-cars-Kolkata_44bbdf61-c591-4f76-8e98-733958ee087a.htm
Error at https://www.cardekho.com/used-car-details/used-Renault-duster-85ps-diesel-rxs-cars-Kolkata_44bbdf61-c591-4f76-8e98-733958ee087a.htm: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=143.0.7499.193)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff60f0288d5
	0x7ff60f028930
	0x7ff60ee0165d
	0x7ff60edfe1dd
	0x7ff60edeeb69
	0x7ff60edf0974
	0x7ff60edef10f
	0x7ff60edee871
	0x7ff60edee5bd
	0x7ff60edec20a
	0x7ff60edec9f2
	0x7ff60ee05950
	0x7ff60eeabad4
	0x7ff60ee81fda
	0x7ff60eeaac97
	0x7ff60ee4ac29
	0x7ff60ee4ba93
	0x7ff60f340620
	0x7ff60f33af60
	0x7ff60f3596c6
	0x7ff60f045dd4
	0x7ff60f04ed7c
	0x7ff60f031ff4
	0x7ff60f0321a5
	0x7ff60f017ed2
	0x7ffdbc59e8d7
	0x7ffdbda0c53c

Scraping 432/945: https://www.cardekho.com/used-car-details/used-Toyota-innova-25-g-diesel-7-seater-bs-iv-cars-Kolkata_

In [13]:
print(car_data)

{'url': 'https://www.cardekho.com/used-car-details/used-Maruti-s-presso-vxi-at-2019-2022-cars-Kolkata_728acab8-3205-4630-87eb-be39ee59fd85.htm', 'Registration Year': '2021', 'Insurance': '-', 'Fuel Type': 'Petrol', 'Seats': '5 Seats', 'Kms Driven': '35,500 Kms', 'RTO': 'Kolkata', 'Ownership': 'First Owner', 'Engine Displacement': '998 cc', 'Transmission': 'Automatic', 'Year of Manufacture': '2021', 'Engine': '998 cc', 'Power': '67.05 bhp', 'Mileage': '21.7 kmpl', 'Fuel Type': 'Petrol', 'No. of Airbags': '1'}
